In [7]:
import pandas as pd
import config_local.local_config as local_config

In [ ]:
train = pd.read_csv(local_config.TRAIN_PROCESS3_CSV)  
test = pd.read_csv(local_config.TEST_PROCESS3_CSV)

In [17]:
to_drop = []#['Street']

In [18]:
train.drop(columns=to_drop, inplace=True, axis=1)

In [ ]:
train.to_csv(__import__("pathlib").Path(local_config.TRAIN_PROCESS4_CSV).resolve().parent / "train_process4.csv", index=False)  
test.to_csv(__import__("pathlib").Path(local_config.TEST_PROCESS4_CSV).resolve().parent / "test_process4.csv", index=False) 